# Importação e preparação dos dados

In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_enunciados.csv', sep = '|')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_ENUNCIADO,TEXTO_ENUNCIADO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995437,Não comprovada a participação do beneficiário ...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025587,A operação ficta de exportação de plataformas ...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455370,"No âmbito do TCU, a dosimetria da pena tem com...",Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773746,O princípio da independência das instâncias pe...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773402,É vedado aos dirigentes das entidades do Siste...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13312, 12)

In [3]:
df.groupby(['DESCR_AREA']).size()

DESCR_AREA
Competência do TCU          557
Contrato Administrativo     942
Convênio                    685
Desestatização              140
Direito Processual         1813
Finanças Públicas           328
Gestão Administrativa       339
Licitação                  2765
Pessoal                    3396
Responsabilidade           2347
dtype: int64

In [4]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [5]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [6]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13312, 10)

In [7]:
from keras.preprocessing.text import Tokenizer
import numpy as np

limite_texto = 200
dim_vetor = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_ENUNCIADO'])

word_index = tokenizer.word_index
vocabulario = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))

sequences = tokenizer.texts_to_sequences(df['TEXTO_ENUNCIADO'])

Using TensorFlow backend.


Found 15387 unique tokens.


In [8]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13312, 200)


In [9]:
from gensim.models import Word2Vec

model = Word2Vec.load('../vocabularios/modelo-acordaos.w2v')

/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
# create a weight matrix for words in training docs

embedding_matrix = np.zeros((vocabulario, 100))

ok = 0
for word, i in tokenizer.word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]
        ok += 1
print('Vocabulario:', i)
print('Encontrados no modelo:', ok, '=', ok * 100. / i)

Vocabulario: 15387
Encontrados no modelo: 14572 = 94.70332098524729


# Treinamento

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU
from keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=False))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0912 11:48:05.097522 140183389796160 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0912 11:48:05.168482 140183389796160 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0912 11:48:05.170251 140183389796160 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0912 11:48:05.177124 140183389796160 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprec

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_1 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2570      
Total params: 1,815,546
Trainable params: 276,746
Non-trainable params: 1,538,800
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 54s 5ms/step - loss: 0.8172 - categorical_accuracy: 0.7270 - val_loss: 0.6635 - val_categorical_accuracy: 0.7830
Epoch 2/20
10649/10649 [==============================] - 53s 5ms/step - loss: 0.3669 - categorical_accuracy: 0.8775 - val_loss: 0.5279 - val_categorical_accuracy: 0.8351

In [12]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 56s 5ms/step - loss: 0.7945 - categorical_accuracy: 0.7405 - val_loss: 0.6597 - val_categorical_accuracy: 0.7845
Epoch 2/20
10649/10649 [==============================] - 55s 5ms/step - loss: 0.3530 - categorical_accuracy: 0.8821 - val_loss: 0.5444 - val_categorical_accuracy: 0.8321
Epoch

In [13]:
from keras.layers import SimpleRNN

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(SimpleRNN(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 256)               91392     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                2570      
Total params: 1,632,762
Trainable params: 1,632,762
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 24s 2ms/step - loss: 1.3703 - categorical_accuracy: 0.5652 - val_loss: 1.7208 - val_categorical_accuracy: 0.4728
Epoch 2/20
10649/10649 [==============================] - 23s 2ms/step - loss: 1.0516 - categorical_accuracy: 0.6608 - val_loss: 1.3563 - val_categorical_accuracy: 0.5805
Epoch

In [14]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2570      
Total params: 1,906,938
Trainable params: 1,906,938
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 69s 6ms/step - loss: 0.7653 - categorical_accuracy: 0.7499 - val_loss: 0.7093 - val_categorical_accuracy: 0.7661
Epoch 2/20
10649/10649 [==============================] - 68s 6ms/step - loss: 0.4179 - categorical_accuracy: 0.8634 - val_loss: 0.5746 - val_categorical_accuracy: 0.8186
Epoch

In [15]:
from keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(SimpleRNN(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               182784    
_________________________________________________________________
dense_5 (Dense)              (None, 10)                5130      
Total params: 1,726,714
Trainable params: 1,726,714
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 35s 3ms/step - loss: 1.3623 - categorical_accuracy: 0.5655 - val_loss: 1.4773 - val_categorical_accuracy: 0.5250
Epoch 2/20
10649/10649 [==============================] - 34s 3ms/step - loss: 1.0944 - categorical_accuracy: 0.6567 - val_loss: 1.4348 - val_categorical_accuracy: 0.5543
Epoch

In [16]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(GRU(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               548352    
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 2,092,282
Trainable params: 2,092,282
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 98s 9ms/step - loss: 0.8075 - categorical_accuracy: 0.7382 - val_loss: 0.6954 - val_categorical_accuracy: 0.7766
Epoch 2/20
10649/10649 [==============================] - 97s 9ms/step - loss: 0.3674 - categorical_accuracy: 0.8762 - val_loss: 0.5424 - val_categorical_accuracy: 0.8092
Epoch

In [17]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 512)               731136    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                5130      
Total params: 2,275,066
Trainable params: 2,275,066
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 122s 11ms/step - loss: 0.7820 - categorical_accuracy: 0.7480 - val_loss: 0.7948 - val_categorical_accuracy: 0.7503
Epoch 2/20
10649/10649 [==============================] - 121s 11ms/step - loss: 0.4384 - categorical_accuracy: 0.8571 - val_loss: 0.6264 - val_categorical_accuracy: 0.8010
E

In [18]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(128))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_4 (GRU)                  (None, 128)               87936     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 1,628,026
Trainable params: 1,628,026
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 63s 6ms/step - loss: 0.9492 - categorical_accuracy: 0.6941 - val_loss: 0.7340 - val_categorical_accuracy: 0.7597
Epoch 2/20
10649/10649 [==============================] - 62s 6ms/step - loss: 0.4159 - categorical_accuracy: 0.8637 - val_loss: 0.6063 - val_categorical_accuracy: 0.8051
Epoch

In [19]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(64))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 200, 100)          1538800   
_________________________________________________________________
gru_5 (GRU)                  (None, 64)                31680     
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 1,571,130
Trainable params: 1,571,130
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 63s 6ms/step - loss: 1.1885 - categorical_accuracy: 0.6150 - val_loss: 0.9795 - val_categorical_accuracy: 0.6992
Epoch 2/20
10649/10649 [==============================] - 62s 6ms/step - loss: 0.5990 - categorical_accuracy: 0.8100 - val_loss: 0.6956 - val_categorical_accuracy: 0.7743
Epoch

In [20]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(512))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_6 (GRU)                  (None, 512)               941568    
_________________________________________________________________
dense_10 (Dense)             (None, 10)                5130      
Total params: 2,485,498
Trainable params: 2,485,498
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 62s 6ms/step - loss: 0.7678 - categorical_accuracy: 0.7493 - val_loss: 0.6088 - val_categorical_accuracy: 0.8085
Epoch 2/20
10649/10649 [==============================] - 61s 6ms/step - loss: 0.3284 - categorical_accuracy: 0.8924 - val_loss: 0.6600 - val_categorical_accuracy: 0.7983
Epoch

In [21]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_7 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_11 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_12 (Dense)             (None, 10)                1290      
Total params: 1,847,162
Trainable params: 1,847,162
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 64s 6ms/step - loss: 0.8139 - categorical_accuracy: 0.7325 - val_loss: 0.6270 - val_categorical_accuracy: 0.7905
Epoch 2/20
10649/10649 [====================

In [22]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_8 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_13 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_14 (Dense)             (None, 10)                650       
Total params: 1,830,074
Trainable params: 1,830,074
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 63s 6ms/step - loss: 0.7926 - categorical_accuracy: 0.7390 - val_loss: 0.6765 - val_categorical_accuracy: 0.7762
Epoch 2/20
10649/10649 [====================

In [23]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(32, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_9 (GRU)                  (None, 256)               274176    
_________________________________________________________________
dense_15 (Dense)             (None, 32)                8224      
_________________________________________________________________
dense_16 (Dense)             (None, 10)                330       
Total params: 1,821,530
Trainable params: 1,821,530
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 61s 6ms/step - loss: 0.8578 - categorical_accuracy: 0.7187 - val_loss: 0.6727 - val_categorical_accuracy: 0.7736
Epoch 2/20
10649/10649 [====================

In [24]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

W0912 16:22:48.239571 140183389796160 deprecation.py:506] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_10 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_17 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 74s 7ms/step - loss: 0.9276 - categorical_accuracy: 0.6929 - val_loss: 0.6795 - val_categorical_accuracy: 0.7833
Epoch 2/20
10649/10649 [==============================] - 72s 7ms/step - loss: 0.4792 - categorical_accuracy: 0.8394 - val_loss: 0.5371 - val_categorical_accuracy: 0.8198
Epoch

In [25]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_11 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_18 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 70s 7ms/step - loss: 0.9398 - categorical_accuracy: 0.6935 - val_loss: 0.7075 - val_categorical_accuracy: 0.7661
Epoch 2/20
10649/10649 [==============================] - 68s 6ms/step - loss: 0.4783 - categorical_accuracy: 0.8397 - val_loss: 0.6032 - val_categorical_accuracy: 0.8044
Epoch

In [26]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_12 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_19 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 64s 6ms/step - loss: 0.7930 - categorical_accuracy: 0.7392 - val_loss: 0.6274 - val_categorical_accuracy: 0.7882
Epoch 2/20
10649/10649 [==============================] - 63s 6ms/step - loss: 0.3312 - categorical_accuracy: 0.8892 - val_loss: 0.6424 - val_categorical_accuracy: 0.7916
Epoch

In [27]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adadelta',  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_13 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_20 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 64s 6ms/step - loss: 0.8135 - categorical_accuracy: 0.7261 - val_loss: 0.6799 - val_categorical_accuracy: 0.7747
Epoch 2/20
10649/10649 [==============================] - 62s 6ms/step - loss: 0.3613 - categorical_accuracy: 0.8772 - val_loss: 0.5257 - val_categorical_accuracy: 0.8175
Epoch

In [28]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, return_sequences=True))
model.add(GRU(32))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_14 (GRU)                 (None, 200, 256)          274176    
_________________________________________________________________
gru_15 (GRU)                 (None, 32)                27744     
_________________________________________________________________
dense_21 (Dense)             (None, 10)                330       
Total params: 1,841,050
Trainable params: 1,841,050
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 121s 11ms/step - loss: 0.7940 - categorical_accuracy: 0.7450 - val_loss: 0.7957 - val_categorical_accuracy: 0.7484
Epoch 2/20
10649/10649 [==================

In [29]:
from keras.layers.core import Dropout

model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256, dropout=0.1, recurrent_dropout=0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=32, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_16 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_22 (Dense)             (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                1290      
Total params: 1,847,162
Trainable params: 1,847,162
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 71s 7ms/step - loss: 1.09

In [30]:
model = Sequential()
model.add(Embedding(vocabulario, dim_vetor, weights=[embedding_matrix], input_length=limite_texto, trainable=True))
model.add(GRU(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=["categorical_accuracy"])
model.summary()

history = model.fit(x, y, epochs=20, batch_size=64, validation_split=0.2, verbose=1, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 200, 100)          1538800   
_________________________________________________________________
gru_17 (GRU)                 (None, 256)               274176    
_________________________________________________________________
dense_24 (Dense)             (None, 10)                2570      
Total params: 1,815,546
Trainable params: 1,815,546
Non-trainable params: 0
_________________________________________________________________
Train on 10649 samples, validate on 2663 samples
Epoch 1/20
10649/10649 [==============================] - 33s 3ms/step - loss: 0.8993 - categorical_accuracy: 0.7054 - val_loss: 0.6798 - val_categorical_accuracy: 0.7845
Epoch 2/20
10649/10649 [==============================] - 30s 3ms/step - loss: 0.4033 - categorical_accuracy: 0.8665 - val_loss: 0.5501 - val_categorical_accuracy: 0.8224
Epoch